In [2]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append("/home/td/Documents/sports_predictor")
sys.path.append("/home/td/Documents/sports_predictor/nba")

from process_data import DataManager

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Using TensorFlow backend.


In [2]:
# dm = DataManager(testing = 2000)
# dm.update_raw_datasets()
# dm.build_timeseries(4, False)
# dm.combine_timeseries(4, False)
# x1, x2, y, cols = dm.get_labeled_data(4, False)
# print(x1.shape, x2.shape, y.shape, len(cols))

In [3]:
# np.isnan(x1).astype(int).sum()

In [4]:
# x2_df = pd.DataFrame(data=x2,
#                     columns = cols)
# x2_df.isna().sum().sum()

In [5]:
# x2_df.head(10)

In [3]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)

team_data.shape, player_data.shape

((436, 46), (5534, 49))

In [4]:
team_data.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,score_diff,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg
0,cho,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,"Scotiabank Arena, Toronto, Canada",1,1.0,2019,3,24,240,41,93,0.441,18,41,0.439,15,17,0.882,14,25,39,32,12,3,7,17,115,NaN,0.572,0.538,0.441,0.183,30.4,89.3,52.7,78.0,13.2,5.9,6.5,100.0,126.8,125.7
1,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,cho,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,"Scotiabank Arena, Toronto, Canada",0,-1.0,2019,3,24,240,45,77,0.584,11,26,0.423,13,13,1.000,3,32,35,29,6,11,13,17,114,NaN,0.689,0.656,0.338,0.169,10.7,69.6,47.3,64.4,6.6,21.2,13.6,100.0,125.7,126.8
2,lac,https://www.basketball-reference.com//teams/LA...,Los Angeles Clippers,nyk,https://www.basketball-reference.com//teams/NY...,New York Knicks,"Madison Square Garden (IV), New York, New York",1,11.0,2019,3,24,240,38,86,0.442,15,35,0.429,33,37,0.892,12,30,42,24,7,4,8,19,124,NaN,0.606,0.529,0.407,0.430,27.9,69.8,48.8,63.2,7.4,7.5,7.3,100.0,130.6,119.0
3,nyk,https://www.basketball-reference.com//teams/NY...,New York Knicks,lac,https://www.basketball-reference.com//teams/LA...,Los Angeles Clippers,"Madison Square Garden (IV), New York, New York",0,-11.0,2019,3,24,240,41,86,0.477,9,33,0.273,22,25,0.880,13,31,44,22,5,7,14,24,113,NaN,0.582,0.529,0.384,0.291,30.2,72.1,51.2,53.7,5.3,13.7,12.6,100.0,119.0,130.6
4,hou,https://www.basketball-reference.com//teams/HO...,Houston Rockets,nop,https://www.basketball-reference.com//teams/NO...,New Orleans Pelicans,"Smoothie King Center, New Orleans, Louisiana",1,23.0,2019,3,24,240,42,86,0.488,21,49,0.429,8,9,0.889,13,37,50,24,10,5,17,16,113,NaN,0.628,0.610,0.570,0.105,31.0,71.2,53.2,57.1,10.7,7.8,15.9,100.0,121.3,96.6


In [78]:
player_data['date_str'] = player_data.apply(
            lambda x: str(x['year']).zfill(4) + '-' + str(x['month']).zfill(2) + '-' + str(x['day']).zfill(2), axis=1)
player_data['game_key'] = player_data.apply(
            lambda x: str(sorted([str(x['date_str']), str(x['team_tag']), str(x['opponent_tag'])])), axis=1)
player_data['team_game_key'] = player_data.apply(
            lambda x: str([str(x['date_str']), str(x['team_tag']), str(x['opponent_tag'])]), axis=1)
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year,date_str,game_key,team_game_key
0,6.0,46.0,0.0,0.0,8,108.0,2.0,6.8,0.417,2.0,1.0,1.000,1.0,0.167,0.333,6.0,6.0,1.000,6.0,1.000,NaN,11,37:38,121.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,0.0,0.0,3.0,https://www.basketball-reference.com//players/...,Howard Eisley,-5.0,11.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,2.0,18.8,2.0,3.2,0.637,13.7,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
1,3.0,23.7,0.0,0.0,8,102.0,4.0,14.0,0.250,2.0,1.0,0.333,3.0,0.300,0.200,10.0,7.0,0.875,8.0,0.800,NaN,11,37:36,85.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,3.0,9.5,5.0,https://www.basketball-reference.com//players/...,Jeff Hornacek,0.0,12.0,NaN,1.0,1.5,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,4.0,22.8,7.0,11.6,0.444,23.1,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
2,1.0,13.8,1.0,2.0,8,101.0,6.0,21.6,0.467,7.0,0.0,NaN,0.0,0.000,0.467,15.0,12.0,0.800,15.0,1.000,NaN,11,36:17,94.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,1.0,3.3,5.0,https://www.basketball-reference.com//players/...,Karl Malone,-5.0,26.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,7.0,24.5,7.0,12.0,0.602,38.8,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
3,1.0,8.9,0.0,0.0,8,96.0,2.0,8.4,0.300,1.0,1.0,0.333,3.0,0.600,0.200,5.0,2.0,1.000,2.0,0.400,NaN,11,30:42,93.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,2.0,7.6,4.0,https://www.basketball-reference.com//players/...,Bryon Russell,0.0,5.0,NaN,3.0,5.4,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,1.0,14.5,4.0,7.9,0.425,10.8,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
4,0.0,0.0,1.0,3.8,8,89.0,3.0,20.5,0.000,0.0,0.0,NaN,0.0,0.000,0.000,3.0,1.0,0.250,4.0,1.333,NaN,11,18:48,37.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,1.0,6.2,5.0,https://www.basketball-reference.com//players/...,Greg Ostertag,-6.0,1.0,NaN,2.0,5.8,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,0.0,0.0,4.0,13.0,0.105,12.2,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"


In [79]:
player_data['team_game_key'].value_counts()

['2003-04-19', 'mil', 'njn']    18
['2003-01-17', 'tor', 'njn']    17
['2001-02-22', 'was', 'sac']    17
['2009-02-23', 'sac', 'noh']    17
['2010-02-16', 'dal', 'okc']    16
                                ..
['2009-02-20', 'sac', 'mem']     9
['2003-04-16', 'tor', 'cle']     8
['2003-02-21', 'sea', 'mil']     8
['1999-03-12', 'njn', 'phi']     8
['2009-02-20', 'chi', 'den']     8
Name: team_game_key, Length: 55508, dtype: int64

In [81]:
player_data['team_game_key'].value_counts().describe(percentiles=[.9, .95, .99])

count    55508.000000
mean        12.262467
std          0.552358
min          8.000000
50%         12.000000
90%         13.000000
95%         13.000000
99%         13.000000
max         18.000000
Name: team_game_key, dtype: float64

In [86]:

def parse_float(s):
    try:
        return float(s)
    except:
        pass

def process_minutes_played(s):
    s_split = str(s).split(':')
    if len(s_split) == 2:
        minutes_parsed = parse_float(s_split[0])
        seconds_parsed = parse_float(s_split[1])
        if minutes_parsed and seconds_parsed:
            total_minutes_played = minutes_parsed + (seconds_parsed/60)
            return total_minutes_played

    return 0
    

In [87]:
player_data['mp'] = player_data['mp'].apply(process_minutes_played)

In [88]:
player_data = player_data.sort_values('date_str')

In [92]:
players = set(player_data['player_link'])

period_size = 32
for t in tqdm.tqdm(players):
    player_data.loc[player_data['player_link'] == t, 'mp_trailing_avg_{}'.format(period_size)] = player_data[player_data['player_link'] == t].shift(periods=1).rolling(window=period_size)['mp'].mean()
    

100%|██████████| 2076/2076 [02:18<00:00, 15.00it/s]


In [102]:
team_game_ids = sorted(list(set(player_data['team_game_key'])))
team_game_ids_vc = player_data['team_game_key'].value_counts()

columns = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                            'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                            'ft_pct',
                                            'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                            'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                            'ts_pct',
                                            'usg_pct']

num_of_players = 12

counter = 0
result_dict = dict()
for i in tqdm.tqdm(team_game_ids):
    sub_df = player_data.iloc[counter:counter+team_game_ids_vc[i]]
    sub_df = sub_df.copy()
    assert sub_df.shape[0] == team_game_ids_vc[i]
    
    sub_df = sub_df.sort_values('mp_trailing_avg_32', ascending = False)
    sub_df = sub_df.reset_index()
    
    
    res_arrays = []
    for j in range(num_of_players):
        if j < sub_df.shape[0]:
            s = sub_df.loc[j]
            s = s[columns]
            s = s.values
            res_arrays.append(s)
        else:
            res_arrays.append(np.array([0 for _ in columns]))
        
    result_dict[i] = res_arrays
    break
    
    



  0%|          | 0/55508 [00:00<?, ?it/s]


In [106]:
np.hstack(result_dict["['1997-11-08', 'atl', 'cle']"]).shape

(408,)

In [5]:
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,score_diff,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year
0,13.0,47.8,0.0,0.0,24,119.0,5.0,24.2,0.235,3.0,2.0,0.286,7.0,0.412,0.176,17.0,7.0,1.0,7.0,0.412,"Scotiabank Arena, Toronto, Canada",3,35:22,120.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,3.0,8.9,2.0,https://www.basketball-reference.com//players/...,Kemba Walker,-9.0,15.0,NaN,1.0,4.0,6.0,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,cho,1.0,4.7,8.0,14.7,0.374,26.6,1,2019
1,0.0,0.0,0.0,0.0,24,133.0,2.0,10.3,0.679,7.0,5.0,0.625,8.0,0.571,0.500,14.0,1.0,0.5,2.0,0.143,"Scotiabank Arena, Toronto, Canada",3,33:16,133.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,1.0,3.1,0.0,https://www.basketball-reference.com//players/...,Dwayne Bacon,-12.0,20.0,NaN,1.0,0.0,0.0,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,cho,0.0,0.0,3.0,5.8,0.672,20.0,1,2019
2,1.0,4.7,1.0,2.8,24,126.0,4.0,20.6,0.667,7.0,2.0,0.500,4.0,0.333,0.583,12.0,0.0,NaN,0.0,0.000,"Scotiabank Arena, Toronto, Canada",3,33:13,115.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,2.0,6.3,4.0,https://www.basketball-reference.com//players/...,Miles Bridges,3.0,16.0,NaN,1.0,1.0,1.6,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,cho,2.0,14.3,6.0,11.7,0.667,18.8,1,2019
3,0.0,0.0,0.0,0.0,24,122.0,1.0,8.3,0.833,4.0,2.0,0.500,4.0,0.667,0.667,6.0,0.0,NaN,0.0,0.000,"Scotiabank Arena, Toronto, Canada",3,20:40,155.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,0.0,0.0,3.0,https://www.basketball-reference.com//players/...,Marvin Williams,-2.0,10.0,NaN,1.0,2.0,5.1,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,cho,0.0,0.0,1.0,3.1,0.833,13.0,1,2019
4,1.0,13.6,0.0,0.0,24,132.0,2.0,28.3,0.750,3.0,0.0,NaN,0.0,0.000,0.750,4.0,0.0,NaN,0.0,0.000,"Scotiabank Arena, Toronto, Canada",3,12:07,130.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,2.0,17.2,1.0,https://www.basketball-reference.com//players/...,Bismack Biyombo,-7.0,6.0,NaN,1.0,0.0,0.0,https://www.basketball-reference.com//teams/CH...,Charlotte Hornets,cho,1.0,20.0,4.0,21.4,0.750,18.4,1,2019
